In [1]:
import Pkg
Pkg.activate("..")

  Activating project at `~/Projects/latentplan.jl`


In [2]:
using Test
using PyCall
using Knet
using Debugger: @enter, @bp, @run
using CUDA

if CUDA.functional()
	atype=KnetArray{Float32}
else	
	atype=Array{Float32}
end
cputype=Array{Float32}

Array{Float32}

In [3]:
check(v) = any(isnan.(cputype(v))) || any(isinf.(cputype(v)))

check (generic function with 1 method)

In [4]:
include("datasets/sequence.jl")
include("models/common.jl")
include("models/transformers.jl")
include("models/vqvae.jl")
include("setup.jl")

No module named 'flow'
/home/mehmeteneserciyes/.julia/conda/3/lib/python3.10/site-packages/glfw/__init__.py:912: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
No module named 'carla'
pybullet build time: Dec 17 2022 18:25:17


read_config (generic function with 1 method)

In [5]:
losssum(prediction) = mean(prediction[2] + prediction[3] + prediction[4])

losssum (generic function with 1 method)

In [6]:
function zerograd_embedding(model::VQContinuousVAE)
    model.model.codebook.embedding = value(model.model.codebook.embedding)
    model.model.codebook.ema_count = value(model.model.codebook.ema_count)
    model.model.codebook.ema_w = value(model.model.codebook.ema_w)
end

zerograd_embedding (generic function with 1 method)

In [7]:
model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_3.jld2", "prev_model");

# Batch and nanification

In [8]:
trainer_config = Dict(
    "batch_size" => 512,
    "learning_rate" => 0.0002,
    "betas" => (0.9, 0.95),
    "weight_decay" => 0.1,
    "grad_norm_clip" => 1.0,
    "lr_decay" => false,
)

Dict{String, Any} with 6 entries:
  "betas"          => (0.9, 0.95)
  "batch_size"     => 512
  "learning_rate"  => 0.0002
  "grad_norm_clip" => 1.0
  "lr_decay"       => false
  "weight_decay"   => 0.1

In [9]:
opt_decay = AdamW(lr=trainer_config["learning_rate"], beta1=trainer_config["betas"][1], beta2=trainer_config["betas"][2], gclip=trainer_config["grad_norm_clip"])
opt_no_decay = AdamW(lr=trainer_config["learning_rate"], beta1=trainer_config["betas"][1], beta2=trainer_config["betas"][2], gclip=trainer_config["grad_norm_clip"])

for p in paramlist_decay(model)
    p.opt = clone(opt_decay)
end
for p in paramlist_no_decay(model)
    p.opt = clone(opt_no_decay)
end

# batch

In [8]:
batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_3.jld2", "prev_batch");

In [9]:
joined_inputs, targets, mask, terminals = atype(batch[1]), atype(batch[2]), atype(batch[3]), atype(batch[4]);

In [10]:
loss = @diff losssum(model(joined_inputs, targets, mask, terminals))

T(0.48638025)

In [16]:
for p in paramlist(model)
    update!(p, grad(loss, p))
end

-0.000287451   -0.00141486    -0.00448274  …   -0.000528297   -7.38718f-5
   0.00109328    -0.00240121    -0.0140845       -0.00250097    -0.000146906
   0.000593902    0.000239694    0.00147629       0.000642225   -0.000190077
  -0.000477957   -0.00100225    -0.00400919      -0.000567076   -0.000132916
   8.31211f-5    -0.00241098    -0.0124349       -0.00294992    -0.000548493
  -0.000273197    0.000467646    0.0028253   …    0.00043963    -0.000256153
   0.000126335 

In [11]:
grad(loss, model.model.predict.w)

17×512 Knet.KnetArrays.KnetMatrix{Float32}:
 -0.000287451  -0.00141486   -0.00448274   …  -0.000528297  -7.38718f-5
  0.00109328   -0.00240121   -0.0140845       -0.00250097   -0.000146906
  0.000593902   0.000239694   0.00147629       0.000642225  -0.000190077
 -0.000477957  -0.00100225   -0.00400919      -0.000567076  -0.000132916
  8.31211f-5   -0.00241098   -0.0124349       -0.00294992   -0.000548493
 -0.000273197   0.000467646   0.0028253    …   0.00043963   -0.000256153
  0.000126335  -0.00122064   -0.00625383      -0.000363271   0.000123527
  9.15883f-5    0.000568493  -0.00690085       0.000216035   0.00103184
  0.000821024   0.00183303    0.00335294       0.00105737    0.000756728
  2.69856f-5    0.001194      0.00487685       0.00136439    0.000368385
  0.000819271   0.00285926    0.0105155    …   0.00318067    0.000397132
  0.000780148   0.0238746     0.096714         0.016012     -0.00271618
 -0.00267367    0.00777439    0.0239344        0.00506062    0.00494828
 -0.0034501

In [20]:
Knet.save("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_updated.jld2", "model", model)

# gradient check

In [12]:
probs, labels = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/bce_inputs.jld2", "pred_terminals", "terminals")

(R(KnetArray{Float32, 3}(1,24,512)), K32(1,24,512)[0.0⋯])

In [21]:
prob_param = Param(value(probs))

P(KnetArray{Float32, 3}(1,24,512))

In [60]:
logits = Param((log.(prob_param ./ (1 .- prob_param .+ 1e-8))))

P(KnetArray{Float32, 3}(1,24,512))

In [ ]:
binary_cross_entropy(prob_param, labels)

0.0026641327f0

In [66]:
labels_cpu = Array{Int32}(reshape(labels, :))

12288-element Vector{Int32}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 1

In [67]:
bce(reshape(logits, :), labels_cpu)

0.002664252f0

In [26]:
loprob_param[1,7,114]

1.0f0

In [29]:
log_pos = clip(log.(prob_param), -100.0f0, Float32(Inf))[1,7,114]

0.0f0

In [31]:
log_neg = clip(log.(1 .- prob_param), -100.0f0, Float32(Inf))[1,7,114]

-100.0f0

In [61]:
cross_entropy_loss = @diff binary_cross_entropy(prob_param, labels)

T(0.0026641327)

In [62]:
findall(x->isnan(x), cputype(grad(cross_entropy_loss, prob_param)))

1-element Vector{CartesianIndex{3}}:
 CartesianIndex(1, 7, 114)

In [68]:
bce_loss = @diff bce(reshape(logits, :), labels_cpu)

T(0.002664252)

# step through third

In [42]:
new_batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_3.jld2", "batch");
joined_inputs, targets, mask, terminals = atype(new_batch[1]), atype(new_batch[2]), atype(new_batch[3]), atype(new_batch[4]);

In [43]:
model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_updated.jld2", "model");

In [44]:
v = model;

In [45]:
joined_dimension, t, b = size(joined_inputs)
padded = repeat_broadcast(v.padding_vector, 1, t, b)

if !(terminals === nothing)
    terminal_mask = repeat_broadcast(deepcopy(1 .- terminals), size(joined_inputs, 1), 1, 1)
    joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask)
end
state = joined_inputs[1:v.observation_dim, 1, :]

11×512 Knet.KnetArrays.KnetMatrix{Float32}:
  1.669      0.120672    1.95312    …   0.0218356   0.520214  -0.384414
 -1.70727    0.641554   -1.43685       -1.10089     1.42574    1.08159
 -0.637838   1.28378    -0.175449       0.349101    0.27128    0.52238
  0.550012   0.494199    0.494093       0.463235    0.684658   0.291366
 -1.45945   -0.17242     1.3172         0.962533    1.34699    0.405659
  0.983976  -1.3038      0.617697   …  -0.415012   -0.682918  -0.846424
  1.07545    0.171964   -0.335095      -0.0970336  -0.62242    0.0143702
 -0.839263   0.0313428  -0.588259      -0.929411   -2.49222   -0.455876
 -0.395215   0.35426     0.260614      -0.965969   -1.88022   -0.163415
  0.704481   0.0849387   0.0846567      0.146934   -0.275631  -0.226125
  0.445358   0.0386078  -0.20211    …   0.164299   -0.215335   0.289442

In [46]:
joined_inputs = cat(joined_inputs, terminals, dims=1);
state = state;

In [54]:
value(v.embed.w)

512×17 Knet.KnetArrays.KnetMatrix{Float32}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN 

In [47]:
v = model.model;

In [50]:
token_embeddings = v.embed(joined_inputs)

512×24×512 KnetArray{Float32, 3}:
[:, :, 1] =
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  Na

# step through second

In [ ]:
v = model;

In [ ]:
pred_trajectory = reshape(reconstructed[1:end-1, :, :], (joined_dimension, t, b))

In [ ]:
pred_terminals = reshape(reconstructed[end, :, :], 1,size(reconstructed)[2:end]...)

In [ ]:
weights = cat(
            atype(ones(2)) .* v.position_weight,
            atype(ones(v.observation_dim - 2)),
            atype(ones(v.action_dim)) .* v.action_weight,
            atype(ones(1)) .* v.reward_weight,
            atype(ones(1)) .* v.value_weight,
            dims=1
        )

In [ ]:
mse = mse_loss(pred_trajectory, joined_inputs, reduction="none") .* reshape(weights, (:, 1, 1))

In [ ]:
function binary_cross_entropy(probs,labels; reduction="mean")
    log_pos = atype(clamp.(cputype(log.(probs)), -100.0f0, Inf))
    log_neg = atype(clamp.(cputype(log.(1 .- probs)), -100.0f0, Inf))
    loss = -(labels .* log_pos .+ (1 .- labels) .* log_neg)
    if reduction == "mean"
        mean(loss)
    elseif reduction == "sum"
        sum(loss)
    elseif reduction == "none"
        return loss
    end
end

In [ ]:
check(mse)

In [ ]:
cross_entropy = binary_cross_entropy(pred_terminals, atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0)))

In [ ]:
labels=atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0));

In [ ]:
labels .* log.(pred_terminals)

In [ ]:
x = (1 .- labels) .* log.(1 .- pred_terminals);

In [ ]:
pred_terminals[1,7,114]

In [ ]:
?clamp

In [ ]:
log_clamp(x) = log(x) < -100.0f0 ? -100.0f0 : log(x)

In [ ]:
clamp.(pred_terminals, -100.0f0, Inf)

In [ ]:
log_clamp.(1 .- pred_terminals)

In [ ]:
findall(x->isnan(x), cputype(x))

# step through

In [ ]:
v = model;

In [ ]:
joined_dimension, t, b = size(joined_inputs)
padded = repeat_broadcast(v.padding_vector, 1, t, b)

if !(terminals === nothing)
    terminal_mask = repeat_broadcast(deepcopy(1 .- terminals), size(joined_inputs, 1), 1, 1)
    joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask)
end
state = joined_inputs[1:v.observation_dim, 1, :]

In [ ]:
joined_inputs = cat(joined_inputs, terminals, dims=1);
state = state;

In [ ]:
check(joined_inputs)

In [ ]:
v = model.model;

In [ ]:
token_embeddings = v.embed(joined_inputs)
position_embeddings = v.pos_emb[:, 1:t, :] 
x_emb = v.drop(token_embeddings .+ position_embeddings)

In [ ]:
check(x_emb)

In [ ]:
x_enc = v.encoder(x_emb)

## Attention

In [ ]:
c=v.encoder.layers[1].attn;
x=ln1_out;

In [ ]:
C, T, B = size(ln1_out)

In [ ]:
k = permutedims(reshape(c.key(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
q = permutedims(reshape(c.query(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
v = permutedims(reshape(c.value(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
att = bmm(permutedims(k, (2,1,3,4)), q) .* Float32(1 / sqrt(size(k, 1)));

In [ ]:
att_masked = att .+ repeat_broadcast(atype((c.mask[1:T,1:T] .== 0) * Float32(-Inf)), 1,1,size(att)[end-1:end]...);

In [ ]:
function softmax(w; dims)
    w = w .- maximum(w, dims=dims)
    probs = exp.(w)
    return probs ./ sum(probs, dims=dims)
end

In [ ]:
att_softmax = softmax(att_masked, dims=1)

In [ ]:
att_masked[:,8,1,1]

In [ ]:
att_softmax[:,8,1,1]

## Rest

In [ ]:
x_pool = permutedims(v.latent_pooling(permutedims(x_enc, (2, 1, 3))), (2,1,3))

In [ ]:
x = v.cast_embed(x_pool)

In [ ]:
check(x)